In [1]:
print("Importing packages...")
import numpy as np
import torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras import preprocessing as k_preproc
from torch.nn import CrossEntropyLoss

Importing packages...


In [2]:
print("Loading tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
print("Loading pretrained model...")
model = GPT2LMHeadModel.from_pretrained('gpt2')

VOCAB_SIZE = tokenizer.vocab_size
USE_GPU = True

TESTING_DATA = "Data/manually_labelled_gsm_data.csv"
TEST_BATCH_SIZE = 1
EPOCH_OF_MODEL_TO_LOAD = 26



PATH_TO_MODEL = f"../Experiment_2/Saved_models/1st_train_GPT2/GPT2_{EPOCH_OF_MODEL_TO_LOAD - 1}epochs.pth"
MAX_LEN = 500
MAX_INPUT_LEN = 450

test = pd.read_csv(TESTING_DATA)

tokenizer.add_special_tokens({'pad_token': '<pad>'})
model.resize_token_embeddings(len(tokenizer))

Loading tokenizer...
Loading pretrained model...


Embedding(50258, 768)

In [3]:
def generate_test_data(dataframe):
    inputs = dataframe['Input'].values
    targets = dataframe['GSM'].values
    corrected_labels = targets
    corrected_inputs = inputs
    return corrected_inputs, corrected_labels

def tokenize_test_sequences(x,y):
	tokenized_x = []
	tokenized_y = []
	for _x,_y in zip(x,y):
		if(type(_x) == str):
			encoded_x = tokenizer.encode(_x)
			encoded_y = tokenizer.encode("LALA")
			dim_x = len(encoded_x)
			if(dim_x <= MAX_LEN):
				tokenized_x.append(encoded_x)
				tokenized_y.append(encoded_y)
	excluded = len(x) - len(tokenized_x)
	print("{} values were excluded because exceed a MAX LENGTH of: {}".format(excluded,MAX_LEN))
	return tokenized_x, tokenized_y

def create_tensors(data):
    tensor_list = []
    max_len_tr = 0
    for elem in data:
        ids = torch.tensor(elem).unsqueeze(0)
        tensor_list.append(ids)
    return tensor_list


In [ ]:

results = []
with torch.no_grad():
    for it, elem in enumerate(tqdm(testloader)):

        inputs = elem
        target_sequence = inputs[0,0].tolist()
        inputs = inputs.to(device)
        predicted_token = 0
        generated_sequence = inputs[0,0].tolist()
        # Computing output tensors
        # Maximizing last tensor (predicted_token_tensor)
        # Concatenating to the input sequence
        # Appending [prediction, target] to results
        while(predicted_token != tokenizer.encode("_") and predicted_token != tokenizer.pad_token_id 
              and len(generated_sequence) < MAX_LEN):
            out = model(inputs)[0][0,0]
            last_tensor = out[-1]
            predicted_token_tensor = torch.argmax(last_tensor)
            predicted_token = predicted_token_tensor.item()
            inputs = torch.cat((inputs,predicted_token_tensor.view(1,1,1)), dim=-1)
            generated_sequence.append(predicted_token)
        results.append(generated_sequence)





In [ ]:
print("Generating text results...")
text_results = []
for i in range(len(results)):
    elem = results[i]
    pred = tokenizer.decode(elem)
    text_results.append([gsms[i],pred])

print("Saving dataframes...")
result_frame = pd.DataFrame(text_results, columns = ['GSM','Prediction'])
result_frame.to_csv(f"Results/manually_labelled_gsm_results_{EPOCH_OF_MODEL_TO_LOAD}_epochs.csv")
# test['Output'] = text_results
# test.to_csv("Results/GPT2/unlabelled_results_16-24epochs_onencode_and_geo_with_gsm.csv")